# Convolutional neural network for the human activities recognition
* using preprocessed data in shape (samples, window-shape, components) to train and test the model
* analyzes motor activities based on signals from sensors
* its six activities: walking, standing, sitting, laying, walking up and downstairs

In [96]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tensorflow import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, Dropout, Flatten
%matplotlib inline

# Loading and reshaping data

In [85]:
# Loading from the .txt files
X_train = np.loadtxt('GeneralData/X_train.txt')
X_test = np.loadtxt('GeneralData/X_test.txt')
y_train = np.loadtxt('GeneralData/y_train.txt')
y_test = np.loadtxt('GeneralData/y_test.txt')

In [86]:
# Shape of X and y data before reshaping
X_train.shape, y_train.shape

((7279, 1152), (7279,))

In [87]:
# Reshaping y data
y_test = np.reshape(y_test,(y_test.shape[0],1))
y_train = np.reshape(y_train,(y_train.shape[0],1))
# Changing the type (int) and decrease the indexes by one - to fit it with binary class matrix (output)
y_test = y_test.astype(int) - 1
y_train = y_train.astype(int) - 1

In [88]:
# Converts an y vectors to binary class matrix
y_test_cat = keras.utils.to_categorical(y_test)
y_train_cat = keras.utils.to_categorical(y_train)

In [92]:
# Visualisating output
y_test_cat[:5]

array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [93]:
# Reshaping X data into (samples, windows-shape, compontents) 
# values from the SingalPreprocessing
samples = 10399
window_shape = 128
components = 9

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1] // components, components))
X_train = np.reshape (X_train, (X_train.shape[0], X_train.shape[1] // components, components))

In [94]:
# X after reshaping
X_train.shape, X_test.shape

((7279, 128, 9), (3120, 128, 9))

* Data is prepared to use into training process

# Modeling CNN
* fitting
* testing and evaluating

In [145]:
# Model creating
model = Sequential()
# Adding layers

# first convolution, 128 filters in size 5
model.add(Conv1D(filters=128, kernel_size=5, input_shape=X_train.shape[1:], activation='relu'))
# second convolution, 64 filters in size 5
model.add(Conv1D(64, 5, activation='relu'))
# max pooling size 2
model.add(MaxPooling1D(2))
# only 25% of data goes on
model.add(Dropout(0.25))
# flattering the matrix
model.add(Flatten())
# deeply connected layer with 150 units
model.add(Dense(units=120, activation='relu'))
# only 50% of data goes on
model.add(Dropout(0.5))
# deeply connected layer with 6 units = 6 activities
model.add(Dense(units=6, activation='softmax'))
# Model Visualisation
model.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_78 (Conv1D)          (None, 124, 128)          5888      
                                                                 
 conv1d_79 (Conv1D)          (None, 120, 64)           41024     
                                                                 
 max_pooling1d_39 (MaxPoolin  (None, 60, 64)           0         
 g1D)                                                            
                                                                 
 dropout_64 (Dropout)        (None, 60, 64)            0         
                                                                 
 flatten_39 (Flatten)        (None, 3840)              0         
                                                                 
 dense_78 (Dense)            (None, 120)               460920    
                                                     